### Part I Review the online programming. 

In [1]:
from collections import defaultdict

original_price = [1, 5, 8, 9, 10, 17, 17, 20, 24, 30, 35]
prices = defaultdict(int)

for i,k in enumerate(original_price):
    prices[i+1] = k

In [2]:
prices

defaultdict(int,
            {1: 1,
             2: 5,
             3: 8,
             4: 9,
             5: 10,
             6: 17,
             7: 17,
             8: 20,
             9: 24,
             10: 30,
             11: 35})

如果想统计 r(n) 的调用次数，同时不需要修改 r(n) 的内部代码，可以使用装饰器(Decorator)

In [3]:
from functools import wraps

In [65]:
called_time_with_arg = defaultdict(int)
solution = {}

In [66]:
def memo(f):
    memo.already_computed = {}
    @wraps(f)
    def _wrap(arg):
        result = None
        
        if arg in memo.already_computed:
            result = memo.already_computed[arg]
        else:
            result = f(arg)
            memo.already_computed[arg] = result
        return result
    return _wrap

In [67]:
@memo
def r(n):
    """
    Args: n is the iron length
    Return: the max revenue
    """
    max_price, max_split = max(
        [(prices[n], 0)] + [(r(i) + r(n-i), i) for i in range(1, n)], 
        key = lambda x:x[0] 
    )
    
    solution[n] = (n-max_split, max_split)
    
    return max_price

In [68]:
r(9)

25

In [69]:
solution

{1: (1, 0),
 2: (2, 0),
 3: (3, 0),
 4: (2, 2),
 5: (3, 2),
 6: (6, 0),
 7: (6, 1),
 8: (6, 2),
 9: (6, 3)}

In [70]:
def parse_solution(n):
    left_split, right_split = solution[n]
    
    if right_split == 0: return [left_split]
    
    return parse_solution(left_split) + parse_solution(right_split)

In [72]:
parse_solution(9)

[6, 3]

### Part 3: Finish the Solution Parse Part of Edit-Distance

operation: DEL、ADD、SUB

In [80]:
solution = {}

In [81]:
def lru_cache(maxsize):
    cache = {}
    def middle(func):
        def _wrap(str1, str2):
            if (str1, str2) in cache: 
                result = cache[(str1, str2)]
            else:
                result = func(str1, str2)
                cache[(str1, str2)] = result
            return result
        return _wrap
    return middle

In [82]:
@lru_cache(maxsize=2**10)
def edit_distance(string1, string2):
    
    if len(string1) == 0: return len(string2)
    if len(string2) == 0: return len(string1)
    
    tail_s1 = string1[-1]
    tail_s2 = string2[-1]
    
    candidates = [
        (edit_distance(string1[:-1], string2) + 1, 'DEL {}'.format(tail_s1)),
        (edit_distance(string1, string2[:-1]) + 1, 'ADD {}'.format(tail_s2)),
    ]
    
    if tail_s1 == tail_s2:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 0, '')
    else:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 1, 'SUB {} => {}'.format(tail_s1, tail_s2))
    
    candidates.append(both_forward)
    
    min_distance, operation = min(candidates, key=lambda x: x[0])
    
    solution[(string1, string2)] = operation
    
    return min_distance

In [83]:
edit_distance('ABCDE', 'ABCCEF')

2

In [84]:
solution

{('A', 'A'): '',
 ('A', 'AB'): 'ADD B',
 ('A', 'ABC'): 'ADD C',
 ('A', 'ABCC'): 'ADD C',
 ('A', 'ABCCE'): 'ADD E',
 ('A', 'ABCCEF'): 'ADD F',
 ('AB', 'A'): 'DEL B',
 ('AB', 'AB'): '',
 ('AB', 'ABC'): 'ADD C',
 ('AB', 'ABCC'): 'ADD C',
 ('AB', 'ABCCE'): 'ADD E',
 ('AB', 'ABCCEF'): 'ADD F',
 ('ABC', 'A'): 'DEL C',
 ('ABC', 'AB'): 'DEL C',
 ('ABC', 'ABC'): '',
 ('ABC', 'ABCC'): 'ADD C',
 ('ABC', 'ABCCE'): 'ADD E',
 ('ABC', 'ABCCEF'): 'ADD F',
 ('ABCD', 'A'): 'DEL D',
 ('ABCD', 'AB'): 'DEL D',
 ('ABCD', 'ABC'): 'DEL D',
 ('ABCD', 'ABCC'): 'SUB D => C',
 ('ABCD', 'ABCCE'): 'ADD E',
 ('ABCD', 'ABCCEF'): 'ADD F',
 ('ABCDE', 'A'): 'DEL E',
 ('ABCDE', 'AB'): 'DEL E',
 ('ABCDE', 'ABC'): 'DEL E',
 ('ABCDE', 'ABCC'): 'DEL E',
 ('ABCDE', 'ABCCE'): '',
 ('ABCDE', 'ABCCEF'): 'ADD F'}

In [91]:
def parse_solution_edit_distance(str1, str2):
    operator = solution[(str1, str2)]
    if 'ADD' in operator: 
        return '{}->{}'.format(operator, parse_solution_edit_distance(str1, str2[:-1]))
    elif 'DEL' in operator:
        return '{}->{}'.format(operator, parse_solution_edit_distance(str1[:-1], str2))
    elif 'SUB' in operator:
        return '{}->{}'.format(operator, parse_solution_edit_distance(str1[:-1], str2[:-1]))
    else:
        return 'finished'

In [93]:
parse_solution_edit_distance('AB', 'ABCCE')

'ADD E->ADD C->ADD C->finished'

### Part 5: Answer following questions:

#### 1. Why do we need dynamic programming? What's the difference of dynamic programming and previous talked search problme?

ANS:

#### 2.Why do we still need dynamic programming? Why not we train a machine learning to fit a function which could get the right answer based on inputs?

ANS:

##### 3.Can you catch up at least 3 problems which could solved by Dynamic Programming?

ANS:

##### 4.Can you catch up at least 3 problems wich could sloved by Edit Distance?

ANS:

#### 5.Please summarize the three main features of Dynamic Programming, and make a concise explain for each feature.

ANS:

#### 6.What's the disadvantages of Dynamic Programming? (You may need search by yourself in Internet)

ANS: